In [11]:
pip install gensim==3.8.3

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\ProgramData\Anaconda3\envs\tf-gpu\python.exe -m pip install --upgrade pip' command.


In [12]:
import re
import os
import numpy as np
import pandas as pd
from pprint import pprint

from konlpy.tag import Okt
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from tqdm import trange

import sqlite3

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
conn = sqlite3.connect("../Precedents.db")
curs = conn.cursor()

sql = "SELECT * FROM Precedents_IC"
curs.execute(sql)

fetchedData = curs.fetchall()
length = len(fetchedData)
fetchedData = [list(fetchedData[x]) for x in range(length)]

In [14]:
data_word=[]

for x in trange(length):
    okt = Okt()
    
    filepath = "C:/Temp/Cases/판례내용/{}.txt".format(fetchedData[x][0])
    if os.path.getsize(filepath) != 0:
        with open(filepath) as file:
            doc = file.read()
        try:
            data_word.append(okt.nouns(doc))
        except Exception as e:
            continue

100%|██████████| 317/317 [01:51<00:00,  2.84it/s]


In [15]:
print(data_word[300])

['피고인', '상고인', '피고인', '및', '검사', '변호인', '변호사', '황', '선철', '외', '인', '원심판결', '전주', '지법', '선고', '노', '판결', '주문', '상고', '모두', '기각', '이유', '먼저', '검사', '상고이유', '중순', '경', '자', '자', '각', '공무집행방해', '점', '대하', '형법', '제', '규정', '공무집행방해', '죄', '공무원', '직무집행', '경우', '성립', '것', '여기', '공무', '집행', '함', '그', '행위', '공무원', '추상', '권한', '속', '뿐', '구체', '직무집행', '관', '법률상', '요건', '방식', '경우', '것이므', '적법성', '결여', '직무', '행위', '공무원', '대항', '폭행', '협박', '공무집행방해', '죄', '수', '대법원', '선고', '도', '판결', '참조', '원심판결', '이유', '원심', '그', '채용', '증거', '종합', '군산', '경찰서', '방범', '소속', '경찰관', '그', '각', '일시', '주류', '판매', '여부', '확인', '위', '피고인', '운영', '노래', '연습장', '검색', '사실', '피고인', '이', '항의', '경찰관', '위', '각', '검색', '업무', '방해', '사실', '각', '인정', '다음', '경찰관', '주류', '판매', '여부', '확인', '위', '노래', '연습장', '검색', '행위', '풍속영업', '규제', '관', '법률', '제', '제', '항', '규정', '검사', '해당', '또', '일반', '허용', '법령', '법관', '발부', '영장', '노래', '연습장', '업주', '의사', '행', '수', '것', '위', '경찰관', '피고인', '의사', '반함', '불구', '영장', '행', '기록', '위', '경찰관', '위', '각', '행위', '직무집

In [16]:
id2word=corpora.Dictionary(data_word)
id2word.filter_extremes(no_below = 2, no_above=0.5)
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

os.environ.update({'MALLET_HOME':r'C:/Temp/mallet-2.0.8/'})
mallet_path = 'C:/Temp/mallet-2.0.8/bin/mallet.bat'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus = corpus, num_topics = 10, id2word = id2word)

In [17]:
#coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
#coherence_ldamallet = coherence_model_ldamallet.get_coherence()

fetchedData_df = pd.DataFrame(fetchedData, columns = ['판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '사건종류명', '사건종류코드', '판결유형', '선고', '판례상세링크'])

def compute_coherence_values(dictionary, corpus, texts, start, limit):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_word, start=1, limit=11)

limit=11; start=1
x = range(start, limit)
topic_num = 0
count = 0
max_coherence = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", cv)
    coherence = cv
    if coherence >= max_coherence:
        max_coherence = coherence
        topic_num = m
        model_list_num = count   
    count = count+1
        
optimal_model = model_list[model_list_num]
model_topics = optimal_model.show_topics(formatted=False)

def format_topics_sentences(ldamodel=optimal_model, corpus=corpus):

    sent_topics_df = pd.DataFrame()

    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)

        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    sent_topics_df = pd.concat([sent_topics_df, fetchedData_df['판례일련번호'], fetchedData_df['사건명'], fetchedData_df['선고일자']], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus)

df_topic_cases = df_topic_sents_keywords.reset_index()
df_topic_cases.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', '판례일련번호', '사건명', '선고일자']
df_topic_cases = df_topic_cases.sort_values(by=['Dominant_Topic'])

# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], axis=0)

sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", '판례일련번호', '사건명', '선고일자', "Num_Documents", "Perc_Documents"]
lda_inform = lda_inform[["Topic_Num", "Keywords", "Num_Documents","Perc_Documents"]]
lda_inform

lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
df_topic_cases['Dominant_Topic'] =df_topic_cases['Dominant_Topic'] +1
df_topic_cases.Dominant_Topic = df_topic_cases.Dominant_Topic.astype(str)
df_topic_cases['Dominant_Topic'] =df_topic_cases['Dominant_Topic'].str.split('.').str[0]

Num Topics = 1  has Coherence Value of 0.2936622423498675
Num Topics = 2  has Coherence Value of 0.48747892469455995
Num Topics = 3  has Coherence Value of 0.48610291035362185
Num Topics = 4  has Coherence Value of 0.49800781799863253
Num Topics = 5  has Coherence Value of 0.5546145210617348
Num Topics = 6  has Coherence Value of 0.5491729099113339
Num Topics = 7  has Coherence Value of 0.5849608201129924
Num Topics = 8  has Coherence Value of 0.5850669807233602
Num Topics = 9  has Coherence Value of 0.5580811900986525
Num Topics = 10  has Coherence Value of 0.5591430675832847


C:\Users\FoodDeuK\AppData\Local\Temp\ipykernel_9116\3295783991.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


<class 'pandas.core.frame.DataFrame'>


In [18]:
lda_inform.to_csv ("./lda_inform_17th.csv", index = None)
lda_inform

,Topic_Num,Keywords,Num_Documents,Perc_Documents
0.0,1,"광고, 프로그램, 카페, 주식회사, 게시, 글, 광고주, 전화, 항의, 중단",18,0.0570
1.0,2,"개인정보, 제공, 진술, 누설, 원, 처리, 당원, 자, 업무, 명부",42,0.1329
2.0,3,"국가정보원, 활동, 보고, 직원, 지시, 법인, 팀, 진술, 국가정보원장, 순번",4,0.0127
3.0,4,"범행, 차, 콜센터, 양형, 상담원, 조직, 사이트, 번호, 범죄수익, 금액",18,0.0570
4.0,5,"개인정보, 회사, 제공, 동의, 게임, 피고, 자, 원고, 수집, 이용자",46,0.1456
5.0,6,"상고이유, 파기, 촬영, 법, 음란물, 청소년, 상고, 타인, 보호, 동영상",124,0.3924
6.0,7,"주식회사, 적시, 표현, 비방, 작성, 후보자, 이익, 글, 기사, 허위",62,0.1962
7.0,8,"국익, 국정원, 지시, 지원, 차장, 단체, 직원, 자금, 전략, 직무",2,0.0063


In [20]:
for i in range(1,topic_num+1):
    globals()['df_{}'.format(i)]=df_topic_cases.loc[df_topic_cases.Dominant_Topic==str(i)]
    globals()['df_{}'.format(i)].sort_values('Topic_Perc_Contrib',ascending=False,inplace = True)
    globals()['df_{}'.format(i)].to_csv ("./topic("+str(i)+")_cases_17th.csv", index = None)

C:\Users\FoodDeuK\AppData\Local\Temp\ipykernel_9116\1026228754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_{}'.format(i)].sort_values('Topic_Perc_Contrib',ascending=False,inplace = True)


In [21]:
from gensim.models import LdaModel

def mallet_to_lda(optimal_model):
    model_gensim = LdaModel(
        id2word=optimal_model.id2word, num_topics=optimal_model.num_topics,
        alpha=optimal_model.alpha, eta=0, iterations=1000,
        gamma_threshold=0.001,
        dtype=np.float32
    )
    model_gensim.sync_state()
    model_gensim.state.sstats = optimal_model.wordtopics
    return model_gensim
model = mallet_to_lda(optimal_model)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word)
pyLDAvis.save_html(vis, 'lda_17th.html')

c:\ProgramData\Anaconda3\envs\tf-gpu\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
c:\ProgramData\Anaconda3\envs\tf-gpu\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
c:\ProgramData\Anaconda3\envs\tf-gpu\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
